<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Just-Amphibia" data-toc-modified-id="Just-Amphibia-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Just Amphibia</a></span></li><li><span><a href="#llmga-find-refs" data-toc-modified-id="llmga-find-refs-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>llmga-find-refs</a></span><ul class="toc-item"><li><span><a href="#Run" data-toc-modified-id="Run-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Run</a></span></li></ul></li><li><span><a href="#--WAITING--" data-toc-modified-id="--WAITING---6"><span class="toc-item-num">6&nbsp;&nbsp;</span>--WAITING--</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Summary</a></span></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* Running `LLMGA-find-refs` pipeline on all Amphibia samples

# Var

In [14]:
work_dir = '/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/MG_assembly_amp/LLMGA-find-refs/'

base_dir = '/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/'
bracken_taxIDs = file.path(base_dir, 'LLMGP_bracken_files.tsv')
metadata_file = '/ebio/abt3_projects/Georg_animal_feces/data/mapping/unified_metadata_complete_190529.tsv'
pipeline_dir = '/ebio/abt3_projects/Georg_animal_feces/bin/llmga-find-refs/'


# Init

In [15]:
library(dplyr)
library(tidyr)
library(ggplot2)
source('/ebio/abt3_projects/Georg_animal_feces/code/misc_r_functions/init.R')

In [26]:
make_dir(dirname(work_dir))
make_dir(work_dir)

Created directory: /ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/MG_assembly_amp 
Created directory: /ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/MG_assembly_amp/LLMGA-find-refs/ 


# Just Amphibia

In [17]:
meta = read.delim(metadata_file, sep='\t') %>%
    dplyr::select(SampleID, class, order, family, genus, scientific_name, diet, habitat)
meta %>% dfhead

[1] 475   8


SampleID,class,order,family,genus,scientific_name,diet,habitat
F14_Common_Bream,Actinopterygii,Cypriniformes,Cyprinidae,Abramis,Abramis_brama,omnivore,aquatic habitat
F35_Red_Deer,Mammalia,Artiodactyla,Cervidae,Cervus,Cervus_elaphus,herbivore,terrestrial habitat
F36_Red_Deer,Mammalia,Artiodactyla,Cervidae,Cervus,Cervus_elaphus,herbivore,terrestrial habitat


In [18]:
taxIDs = read.delim(bracken_taxIDs, sep='\t') %>%
    mutate(Sample = gsub('^XF', 'F', Sample))
taxIDs %>% dfhead

[1] 291   2


Sample,TaxIDs
X101_Horse,/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/LLMGP_sub5k/kraken/X101_Horse_bracken.tsv
X102_Kulan,/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/LLMGP_sub5k/kraken/X102_Kulan_bracken.tsv
X103_Eurasian_Blackcap,/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/LLMGP_sub5k/kraken/X103_Eurasian_Blackcap_bracken.tsv


In [19]:
# joining
taxIDs = taxIDs %>%
    inner_join(meta, c('Sample'='SampleID')) 

taxIDs %>% dfhead

Warning message:
“Column `Sample`/`SampleID` joining character vector and factor, coercing into character vector”

[1] 290   9


Sample,TaxIDs,class,order,family,genus,scientific_name,diet,habitat
X101_Horse,/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/LLMGP_sub5k/kraken/X101_Horse_bracken.tsv,Mammalia,Perissodactyla,Equidae,Equus,Equus_caballus,herbivore,terrestrial habitat
X102_Kulan,/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/LLMGP_sub5k/kraken/X102_Kulan_bracken.tsv,Mammalia,Perissodactyla,Equidae,Equus,Equus_hemionus,herbivore,terrestrial habitat
X103_Eurasian_Blackcap,/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/LLMGP_sub5k/kraken/X103_Eurasian_Blackcap_bracken.tsv,Aves,Passeriformes,Sylviidae,Sylvia,Sylvia_atricapilla,omnivore,terrestrial habitat


In [20]:
# all metadata 
taxIDs %>%
    group_by(class) %>%
    summarize(n = n()) %>%
    ungroup()

class,n
Actinopterygii,27
Amphibia,8
Aves,63
Mammalia,179
Reptilia,13


In [21]:
taxIDs_f = taxIDs %>%
    filter(class == 'Amphibia') 

taxIDs_f %>% dfhead

[1] 8 9


Sample,TaxIDs,class,order,family,genus,scientific_name,diet,habitat
X192_Common_Frog,/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/LLMGP_sub5k/kraken/X192_Common_Frog_bracken.tsv,Amphibia,Anura,Ranidae,Rana,Rana_temporaria,carnivore,terrestrial habitat
X193_fail_Common_Frog,/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/LLMGP_sub5k/kraken/X193_fail_Common_Frog_bracken.tsv,Amphibia,Anura,Ranidae,Rana,Rana_temporaria,carnivore,terrestrial habitat
X194_Agile_Frog,/ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/LLMGP_sub5k/kraken/X194_Agile_Frog_bracken.tsv,Amphibia,Anura,Ranidae,Rana,Rana_dalmatina,carnivore,terrestrial habitat


In [27]:
outF = file.path(work_dir, 'LLMGP_bracken_taxIDs_amp.tsv')
taxIDs_f %>%
    arrange(class, order, family, genus) %>%
    write.table(outF, sep='\t', quote=FALSE, row.names=FALSE)
cat('File written:', outF, '\n')

File written: /ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/MG_assembly_amp/LLMGA-find-refs//LLMGP_bracken_taxIDs_amp.tsv 


# llmga-find-refs

In [29]:
F = file.path(work_dir, 'config.yaml')
cat_file(F)

#-- I/O --#
# table with sample --> read_file information
samples_file: /ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/MG_assembly_amp/LLMGA-find-refs/LLMGP_bracken_taxIDs_amp.tsv

# output location
output_dir:   /ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/MG_assembly_amp/LLMGA-find-refs/

#-- database --#
centrifuge_db: /ebio/abt3_projects/databases_no-backup/centrifuge/p+h+v
taxonomy_db: /ebio/abt3_projects/databases_no-backup/NCBI_genome/NCBI_taxid_db/taxa.sqlite

#-- software parameters --#
params:
  # If providing paired-end reads
  seqtk: 1000000          # subsampled read pairs per sample; use "Skip" to skip
  centrifuge: ""          # for taxonomic profiling unless providing "TaxID" column
  # If providing tables of TaxIDs instead of reads
  taxID_column: taxonomy_id                # column in the table(s) containing the taxIDs
  abundance_column: fraction_total_reads   # column in the 

## Run

```
(snakemake_dev) @ rick:/ebio/abt3_projects/Georg_animal_feces/bin/llmga-find-refs
$ screen -L -S llmga-fr-ga-amp ./snakemake_sge.sh /ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/MG_assembly_amp/LLMGA-find-refs/config.yaml cluster.json /ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/MG_assembly_amp/LLMGA-find-refs/SGE_log 20
```

# Summary

In [30]:
# listing all reference genomes
ref_dir = file.path(work_dir, 'references', 'drep', 'dereplicated_genomes')
F = list.files(ref_dir, '*.fna', full.names=TRUE)
cat('Number of ref. genomes:', F %>% length, '\n')

Number of ref. genomes: 176 


In [31]:
# combining all refs
ref_file = file.path(work_dir, 'references', 'ref_genomes.fna')
if(file.exists(ref_file)){ file.remove(ref_file) }
file.create(ref_file)
for (f in F){
    file.append(ref_file, f)
}
file.size(ref_file)
cat('File written:', ref_file, '\n')

[1] TRUE

[1] 752393851

File written: /ebio/abt3_projects/Georg_animal_feces/data/metagenome/HiSeqRuns126-133-0138/wOutVertebrata/MG_assembly_amp/LLMGA-find-refs//references/ref_genomes.fna 


# sessionInfo

In [32]:
pipelineInfo(pipeline_dir)

LLMGA-find-refs

Ley Lab Metagenome Assembly: Find References (LLMGA-find-refs)

* Version: 0.2.1
* Authors:
  * Nick Youngblut <nyoungb2@gmail.com>
* Maintainers:
  * Nick Youngblut <nyoungb2@gmail.com>

--- conda envs ---
==> /ebio/abt3_projects/Georg_animal_feces/bin/llmga-find-refs//bin/envs/genome_dl.yaml <==
channels: !!python/tuple
- bioconda
dependencies:
- pigz
- python>=3.6
- bioconda::ete3=3.1.1
- bioconda::ncbi-genome-download=0.2.6

==> /ebio/abt3_projects/Georg_animal_feces/bin/llmga-find-refs//bin/envs/genome_drep.yaml <==
channels: !!python/tuple
- bioconda
dependencies:
- pigz
- bioconda::mash
- pip:
  - drep

==> /ebio/abt3_projects/Georg_animal_feces/bin/llmga-find-refs//bin/envs/reads.yaml <==
channels: !!python/tuple
- bioconda
dependencies:
- pigz
- bioconda::seqtk
- bioconda::centrifuge

In [33]:
sessionInfo()

R version 3.4.1 (2017-06-30)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.3 LTS

Matrix products: default
BLAS: /ebio/abt3_projects/software/miniconda3_gt4.4/envs/py3_physeq/lib/R/lib/libRblas.so
LAPACK: /ebio/abt3_projects/software/miniconda3_gt4.4/envs/py3_physeq/lib/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] bindrcpp_0.2.2 ggplot2_3.0.0  tidyr_0.8.1    dplyr_0.7.6   

loaded via a namespace (and not attached):
 [1] Rcpp_0.12.17     bindr_0.1.1      magrittr_1.5     munsell_0.5.0   
 [5] tidyselect_0.2